In [ ]:
import os
import pandas as pd
import json
from openai import OpenAI

client = OpenAI(
    api_key="sk-",
    base_url="https://api.zhizengzeng.com/v1/",  # 中转gpt接口
)

# 情感分类的系统提示（已为二分类）
system_message = '''
你是一个情感分类助手，你将帮助我对每一条数据进行情感分析。
情感类别包括：
1. 积极
2. 消极

请根据内容只输出“积极”或“消极”，不要添加任何其他内容。
例如：
“服务态度很好，十分满意。” → 积极
“工作人员态度很差，非常生气。” → 消极
请用中文输出情感类别。
'''

# 输入和输出文件路径
file_path = "ChnSentiCorp_htl_all.csv"
output_file = "gpt4-mini_CH.csv"

# 读取输入数据
df = pd.read_csv(file_path)

# 若有已处理部分，则断点续算
if os.path.exists(output_file):
    processed_df = pd.read_csv(output_file)
    last_index = processed_df.shape[0]
else:
    processed_df = pd.DataFrame(columns=["label","review", "sentiment"])
    last_index = 0

# API情感分类调用
def classify_sentiment_ali(content):
    try:
        completion = client.chat.completions.create(
            model="gpt-4o-mini",  # 改为你需要的模型名
            messages=[
                {'role': 'system', 'content': system_message},
                {'role': 'user', 'content': f"{content}"}
            ]
        )
        response = json.loads(completion.model_dump_json())
        label = response['choices'][0]['message']['content'].strip()
        # 只保留“积极”或“消极”
        if "积极" in label:
            return "积极"
        elif "消极" in label:
            return "消极"
        else:
            return "未知"
    except Exception as e:
        print(f"API error at: {content}, error: {e}")
        return "API调用失败"

# 处理数据，持续保存
for idx, row in df.iloc[last_index:].iterrows():
    content = row['review']
    sentiment = classify_sentiment_ali(content)
    label = row['label'] 
    processed_df = pd.concat([
        processed_df,
        pd.DataFrame([{"label":label, "review": content, "sentiment": sentiment}])
    ], ignore_index=True)
    processed_df.to_csv(output_file, index=False,encoding='utf-8-sig')
    print(f"已处理第 {idx+1} 条，结果：{sentiment}")

print("情感分类完成，结果已保存。")


已处理第 1 条，结果：积极
已处理第 2 条，结果：消极
已处理第 3 条，结果：消极
已处理第 4 条，结果：消极
已处理第 5 条，结果：积极
已处理第 6 条，结果：积极
已处理第 7 条，结果：消极
已处理第 8 条，结果：积极
已处理第 9 条，结果：消极
已处理第 10 条，结果：积极
已处理第 11 条，结果：消极
已处理第 12 条，结果：积极
已处理第 13 条，结果：积极
已处理第 14 条，结果：消极
已处理第 15 条，结果：消极
已处理第 16 条，结果：消极
已处理第 17 条，结果：消极
已处理第 18 条，结果：消极
已处理第 19 条，结果：积极
已处理第 20 条，结果：积极
已处理第 21 条，结果：消极
已处理第 22 条，结果：积极
已处理第 23 条，结果：消极
已处理第 24 条，结果：积极
已处理第 25 条，结果：积极
已处理第 26 条，结果：消极
已处理第 27 条，结果：消极
已处理第 28 条，结果：消极
已处理第 29 条，结果：消极
已处理第 30 条，结果：消极
已处理第 31 条，结果：消极
已处理第 32 条，结果：消极
已处理第 33 条，结果：消极
已处理第 34 条，结果：消极
已处理第 35 条，结果：积极
已处理第 36 条，结果：消极
已处理第 37 条，结果：积极
已处理第 38 条，结果：消极
已处理第 39 条，结果：消极
已处理第 40 条，结果：消极
已处理第 41 条，结果：消极
已处理第 42 条，结果：积极
已处理第 43 条，结果：消极
已处理第 44 条，结果：消极
已处理第 45 条，结果：积极
已处理第 46 条，结果：消极
已处理第 47 条，结果：消极
已处理第 48 条，结果：消极
已处理第 49 条，结果：消极
已处理第 50 条，结果：积极
已处理第 51 条，结果：积极
已处理第 52 条，结果：消极
已处理第 53 条，结果：积极
已处理第 54 条，结果：消极
已处理第 55 条，结果：消极
已处理第 56 条，结果：消极
已处理第 57 条，结果：积极
已处理第 58 条，结果：消极
已处理第 59 条，结果：积极
已处理第 60 条，结果：消极
已处理第 61 条，结果：积极
已处理第 62 条，结果：消极
已处理第 63 条，结果：积极
已

In [1]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

df = pd.read_csv('gpt4-mini_CH.csv')

# 只保留 sentiment 为“积极”或“消极”的数据
df = df[df['sentiment'].isin(['积极', '消极'])]

# 映射 sentiment 到 0/1
sentiment_map = {'积极': 1, '消极': 0}
df['sentiment_num'] = df['sentiment'].map(sentiment_map)

# 计算各项指标
precision = precision_score(df['label'], df['sentiment_num'])
recall = recall_score(df['label'], df['sentiment_num'])
f1 = f1_score(df['label'], df['sentiment_num'])

print(f'精确率: {precision:.4f}')
print(f'召回率: {recall:.4f}')
print(f'F1分数: {f1:.4f}')

精确率: 0.9793
召回率: 0.7090
F1分数: 0.8225


In [ ]:
import os
import pandas as pd
import json
from openai import OpenAI

client = OpenAI(
    api_key="sk-",
    base_url="https://api.zhizengzeng.com/v1/",  # 中转gpt接口
)
# 情感分类的系统提示
system_message = '''
你是一个情感分类助手，你将帮助我对每一条数据进行情感分析。
情感类别包括：

1. angry（愤怒）
2. fear（害怕）
3. happy（高兴）
4. neutral（中性/无明显情感）
5. sad（悲伤）
6. surprise（惊讶）

请根据内容只输出情感类别的英文标签，不要添加任何其他内容。
例如：
“服务态度很好，十分满意。” → happy
“工作人员态度很差，非常生气。” → angry
“听到这个消息，我感到很害怕。” → fear
“我觉得没什么特别的。” → neutral
“失去了心爱的宠物，我很难过。” → sad
“没想到你会来，真让我吃惊！” → surprise
请用英文标签输出情感类别。
'''

# 输入和输出文件路径
file_path = "usual_eval_labeled.csv"
output_file = "gpt4-mini_US.csv"

# 读取输入数据
df = pd.read_csv(file_path)

# 若有已处理部分，则断点续算
if os.path.exists(output_file):
    processed_df = pd.read_csv(output_file)
    last_index = processed_df.shape[0]
else:
    processed_df = pd.DataFrame(columns=["label","content", "sentiment"])
    last_index = 0

# API情感分类调用
def classify_sentiment_ali(content):
    try:
        completion = client.chat.completions.create(
            model="gpt-4o-mini",  # 改为你需要的模型名
            messages=[
                {'role': 'system', 'content': system_message},
                {'role': 'user', 'content': f"{content}"}
            ],
            extra_body={"enable_thinking": False},
        )
        response = json.loads(completion.model_dump_json())
        label = response['choices'][0]['message']['content'].strip()
        # 六分类输出
        valid_labels = ["angry", "fear", "happy", "neutral", "sad", "surprise"]
        label = label.lower()  # 转小写，防止大小写不一致
        if label in valid_labels:
            return label
        else:
            return "未知"
    except Exception as e:
        print(f"API error at: {content}, error: {e}")
        return "API调用失败"

# 处理数据，持续保存
for idx, row in df.iloc[last_index:].iterrows():
    content = row['content']
    sentiment = classify_sentiment_ali(content)
    label = row['label'] 
    processed_df = pd.concat([
        processed_df,
        pd.DataFrame([{"label":label, "content": content, "sentiment": sentiment}])
    ], ignore_index=True)
    processed_df.to_csv(output_file, index=False,encoding='utf-8-sig')
    print(f"已处理第 {idx+1} 条，结果：{sentiment}")

print("情感分类完成，结果已保存。")

已处理第 1 条，结果：angry
已处理第 2 条，结果：neutral
已处理第 3 条，结果：neutral
已处理第 4 条，结果：fear
已处理第 5 条，结果：neutral
已处理第 6 条，结果：angry
已处理第 7 条，结果：fear
已处理第 8 条，结果：neutral
已处理第 9 条，结果：happy
已处理第 10 条，结果：sad
已处理第 11 条，结果：angry
已处理第 12 条，结果：neutral
已处理第 13 条，结果：neutral
已处理第 14 条，结果：sad
已处理第 15 条，结果：sad
已处理第 16 条，结果：angry
已处理第 17 条，结果：happy
已处理第 18 条，结果：neutral
已处理第 19 条，结果：surprise
已处理第 20 条，结果：fear
已处理第 21 条，结果：angry
已处理第 22 条，结果：neutral
已处理第 23 条，结果：angry
已处理第 24 条，结果：neutral
已处理第 25 条，结果：neutral
已处理第 26 条，结果：neutral
已处理第 27 条，结果：surprise
已处理第 28 条，结果：neutral
已处理第 29 条，结果：surprise
已处理第 30 条，结果：surprise
已处理第 31 条，结果：neutral
已处理第 32 条，结果：angry
已处理第 33 条，结果：happy
已处理第 34 条，结果：angry
已处理第 35 条，结果：angry
已处理第 36 条，结果：angry
已处理第 37 条，结果：sad
已处理第 38 条，结果：sad
已处理第 39 条，结果：fear
已处理第 40 条，结果：happy
已处理第 41 条，结果：sad
已处理第 42 条，结果：surprise
已处理第 43 条，结果：happy
已处理第 44 条，结果：sad
已处理第 45 条，结果：sad
已处理第 46 条，结果：neutral
已处理第 47 条，结果：sad
已处理第 48 条，结果：neutral
已处理第 49 条，结果：neutral
已处理第 50 条，结果：angry
已处理第 51 条，结果：neutral
已处理第 52 条，结果：

In [3]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

df = pd.read_csv('gpt4-mini_US.csv')
df = df[(df['label'] != 'API调用失败') & (df['sentiment'] != 'API调用失败')]
df = df[(df['label'] != '未知') & (df['sentiment'] != '未知')]
# 计算各项指标
precision = precision_score(df['label'], df['sentiment'],average='macro')
recall = recall_score(df['label'], df['sentiment'],average='macro')
f1 = f1_score(df['label'], df['sentiment'],average='macro')

print(f'精确率: {precision:.4f}')
print(f'召回率: {recall:.4f}')
print(f'F1分数: {f1:.4f}')


精确率: 0.7179
召回率: 0.7070
F1分数: 0.6883
